<a href="https://colab.research.google.com/github/Jdka1/Drake-AI/blob/main/Song_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://app.kits.ai/convert/shared/drake-47

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.8/220.8 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

# Data Loading

In [ ]:
!mkdir -p ~/.kaggle
!touch ~/.kaggle/kaggle.json
!ls -a ~/.kaggle

.  ..  kaggle.json


In [ ]:
import json

kaggle_json = {}

with open('/root/.kaggle/kaggle.json', 'w') as f:
  json.dump(kaggle_json, f)

In [ ]:
!kaggle datasets download -d juicobowley/drake-lyrics

100% 764k/764k [00:00<00:00, 888kB/s]
100% 764k/764k [00:00<00:00, 888kB/s]


In [ ]:
!unzip /content/drake-lyrics.zip

Archive:  /content/drake-lyrics.zip
  inflating: drake_data.csv          
  inflating: drake_data.json         
  inflating: drake_lyrics.txt        


# Data Processing

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drake_data.csv")
df

FileNotFoundError: ignored

In [ ]:
df = df.query("lyrics.notna()").reset_index(drop=True)
df.lyrics.isna().sum()
df.to_csv("/content/drake_data.csv", index= False)

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=)

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)

In [ ]:
def lyrics_to_prompt(lyrics):
  prompt = f"Come up with a prompt that would result in the generation of this song.\n\nLyrics:{lyrics}\n\nPrompt:"
  response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": "You are a helpful assistant."},
      {"role": "user", "content": prompt},
    ]
  )
  return response.choices[0].message.content

#print(lyrics_to_prompt(df.lyrics[1]))

In [ ]:
df["prompt"] = df.lyrics.apply(lyrics_to_prompt)

In [ ]:
df.to_csv("drake_with_prompts.csv",index=False)

In [ ]:
from google.colab import files
files.download('drake_with_prompts.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Loading Training Data

In [ ]:
import pandas as pd

csv_url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTjn49YWFG5pzoXyw7f4IU2TXw0vu0pCjHWc3k9qbd9MSADgr4SoGTljMKPblRWYo9xsAjOXD7ytUff/pub?output=csv'
df = pd.read_csv(csv_url)

In [ ]:
df = df.query("lyrics.notna()").reset_index(drop=True)
df.lyrics.isna().sum()

0

In [ ]:
df['prompt'] = df.prompt.apply(lambda prompt: prompt + '\n\n')
df

,Unnamed: 0,album,lyrics_title,lyrics_url,lyrics,track_views,prompt
0,0,Certified Lover Boy,Certified Lover Boy* Lyrics,https://genius.com/Drake-certified-lover-boy-l...,[Verse]\nPut my feelings on ice\nAlways been a...,8.7K,Write a song about a person who presents thems...
1,1,Certified Lover Boy,Like I’m Supposed To/Do Things Lyrics,https://genius.com/Drake-like-im-supposed-to-d...,[Verse]\nHands are tied\nSomeone's in my ear f...,38.8K,Write a song about the complexities of a relat...
2,2,Certified Lover Boy,Not Around Lyrics,https://genius.com/Drake-not-around-lyrics,"[Intro]\nYeah, we back\nWassup ladies?\nSwisha...",129.8K,Write a song about a woman who moves away from...
3,3,Certified Lover Boy,In the Cut (Ft. Roddy Ricch) Lyrics,https://genius.com/Drake-in-the-cut-lyrics,"[Intro: Drake]\nAyy, yeah\nPipe this shit up a...",72.1K,Write a song about rising above adversity and ...
4,4,Certified Lover Boy,Zodiac Sign (Ft. Jessie Reyez) Lyrics,https://genius.com/Drake-zodiac-sign-lyrics,[Verse 1: Drake]\nYou ask how many girls I bee...,54.8K,Write a song that explores the complexities of...
...,...,...,...,...,...,...,...
283,283,Unreleased Songs,Pop Style (Demo) (Ft. Kanye West) Lyrics,https://genius.com/Drake-pop-style-demo-lyrics,Lyrics from cut studio recording\n\n[Verse 1: ...,NaN,Write a song about the importance of family an...
284,284,Unreleased Songs,Time Flies (Demo) (Ft. Future) Lyrics,https://genius.com/Drake-time-flies-demo-lyrics,Lyrics from Snippet\n\n[Chorus: Drake & Future...,NaN,"Write a song about loyalty, friendship, and th..."
285,285,Unreleased Songs,Walk It Talk It (Demo) by Quavo (Ft. Drake) Ly...,https://genius.com/Quavo-walk-it-talk-it-demo-...,"[Intro: Quavo]\nYeah, yeah\n(Deko)\nWoah, hold...",NaN,Write a song about confidence and authenticity...
286,286,Unreleased Songs,God’s Plan (Demo) (Ft. Trippie Redd) Lyrics,https://genius.com/Drake-gods-plan-demo-lyrics,"[Intro: Drake]\nThey wishing, they wishing, th...",NaN,Write a song about facing adversity and stayin...


In [ ]:
# df.iloc[len(df)-1]
df['prompt']

0      Write a song about a person who presents thems...
1      Write a song about the complexities of a relat...
2      Write a song about a woman who moves away from...
3      Write a song about rising above adversity and ...
4      Write a song that explores the complexities of...
                             ...                        
283    Write a song about the importance of family an...
284    Write a song about loyalty, friendship, and th...
285    Write a song about confidence and authenticity...
286    Write a song about facing adversity and stayin...
287    Write a song about overcoming obstacles and pe...
Name: prompt, Length: 288, dtype: object

In [ ]:
df.to_csv("/content/drake_data.csv", index=False)

In [ ]:
for i in pd.read_csv("/content/drake_data.csv")['prompt']:
  print(i)

Write a song about a person who presents themselves as a charming and desirable lover, but deep down, they struggle to truly open up and connect emotionally. Show the contrast between their surface-level charisma and their underlying emotional detachment.


Write a song about the complexities of a relationship where one person is hesitant to fully commit due to past experiences, while the other person remains constant, reassuring their love and commitment.


Write a song about a woman who moves away from her hometown, leaves behind her old life as a dancer, and sets out to start her own business. Despite the challenges and the absence of her parents, she perseveres and finds support from unexpected sources.


Write a song about rising above adversity and achieving success, despite facing challenges along the way. Emphasize the importance of staying true to oneself and not getting caught up in trends or external validation. Incorporate elements of determination, perseverance, and the ce

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
from rich import print
import re
tokenizer = AutoTokenizer.from_pretrained("databricks/dolly-v2-3b")
def tokenize_lyrics(batch):
    # Tokenize the lyrics using the tokenizer
    def join_lyrics (song):
      if isinstance(song, list):
        song = "".join(song)
      pattern = r'\[.*?\]'
      return re.sub(pattern, '', song)
    # lyrics = [join_lyrics(song) for song in batch["lyrics"]]
    lyrics = batch['lyrics']
    tokenized_lyrics = tokenizer(lyrics, padding='max_length', truncation=True, max_length=512)
    tokenized_prompt = tokenizer(batch["prompt"], padding='max_length', truncation=True, max_length=512)

    prompt_tokens = tokenized_prompt['input_ids']
    lyrics_tokens = tokenized_lyrics['input_ids']

    # print(lyrics_tokens[0])
    # print(type(batch))

    input_ids = prompt_tokens + lyrics_tokens
    # print(input_ids[:100])
    # print(lyrics_tokens[0])

    labels = [-100] * len(prompt_tokens) + lyrics_tokens

    return {
        'input_ids': input_ids,
        'labels': labels
    }

    # return {
    #     **{f'lyrics_{key}': val for key, val in tokenized_lyrics.items()}, **{f'prompt_{key}': val for key, val in tokenized_prompt.items()}
    # }

dataset = load_dataset('csv', data_files="/content/drake_data.csv")
dataset = dataset['train'].train_test_split(test_size=0.2, seed=42)
train_dataset = dataset["train"].map(tokenize_lyrics, batched=False)
validation_dataset = dataset["test"].map(tokenize_lyrics, batched=False)

tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/230 [00:00<?, ? examples/s]

Map:   0%|          | 0/58 [00:00<?, ? examples/s]

## Dataset: https://www.kaggle.com/datasets/juicobowley/drake-lyrics

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "databricks/dolly-v2-3b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

config.json:   0%|          | 0.00/819 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.68G [00:00<?, ?B/s]

In [ ]:
model

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50280, 2560)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear4bit(in_features=2560, out_features=7680, bias=True)
          (dense): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear4bit(in_features=2560, out_features=10240, bias=True)
          (dense_4h_to_h): Linear4bit(in_features=10240, out_features=2

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 2621440 || all params: 1519416320 || trainable%: 0.172529409187865

In [ ]:
import transformers


class DrakeTrainer(transformers.Trainer):
  def compute_loss(self, model, inputs):
    return model(**inputs).loss



# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = DrakeTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset = validation_dataset,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=8,
        warmup_steps=4,
        max_steps=15,
        learning_rate=1e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_steps=4,
        eval_steps=4,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,3.277800
2,3.567800
3,3.436400
4,3.430900
5,3.261800
6,3.236000
7,3.132200
8,3.063100
9,2.944000
10,2.514300


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

TrainOutput(global_step=60, training_loss=2.680845681826274, metrics={'train_runtime': 928.8115, 'train_samples_per_second': 0.517, 'train_steps_per_second': 0.065, 'total_flos': 7812191512166400.0, 'train_loss': 2.680845681826274, 'epoch': 2.09})

In [ ]:
prompt = "Write a song that explores the desire for success and the challenges faced along the way. The lyrics should convey a sense of determination, confidence, and a willingness to confront haters and doubters. The song should also touch on the themes of money, fame, and the pursuit of personal fulfillment.\n"
input_ids = tokenizer(prompt, return_tensors = "pt").to("cuda:0")
output = model.generate(**input_ids, do_sample=True, max_length=100)
generated_lyrics = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_lyrics, '\n\n')
generated_lyrics

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Write a song that explores the desire for success and the challenges faced along the way. The lyrics should convey 
a sense of determination, confidence, and a willingness to confront haters and doubters. The song should also touch
on the themes of money, fame, and the pursuit of personal fulfillment.
[Chorus]
I'm a man that never takes no for an answer
And I'll get to where I'm goin'
I won't take no for an answer
I

"Write a song that explores the desire for success and the challenges faced along the way. The lyrics should convey a sense of determination, confidence, and a willingness to confront haters and doubters. The song should also touch on the themes of money, fame, and the pursuit of personal fulfillment.\n[Chorus]\nI'm a man that never takes no for an answer\nAnd I'll get to where I'm goin'\nI won't take no for an answer\nI"

In [ ]:
prompt = generated_lyrics
input_ids = tokenizer(prompt, return_tensors = "pt").to("cuda:0")
output = model.generate(**input_ids, do_sample=True, max_length=900)
generated_lyrics = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_lyrics, '\n\n')
generated_lyrics

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Write a song that explores the desire for success and the challenges faced along the way. The lyrics should convey 
a sense of determination, confidence, and a willingness to confront haters and doubters. The song should also touch
on the themes of money, fame, and the pursuit of personal fulfillment.
[Chorus]
I'm a man that never takes no for an answer
And I'll get to where I'm goin'
I won't take no for an answer
I'll take this right to the top
And I'm doin' it for the money
Yeah, I'm doin' it for the money
[Verse 1]
I've been waiting for my moment
Wish that I'd have gotten a bit more exposure
I coulda taken control
But things happened and shit that just got real
No one recognized me
I had to find my own way
But now I'm on the rise
Takin' over others
But my goal is still the same
I'll hit the mark if it's meant for me to do so
I've been through the grind
I've been through the heartache
But I've always known that I'll get there
I'll get there if I still believe
I haven't been myself in a long time
But now I'm back just getting all the shit done
I've been through the war
And I've got the scars
But I'm still comin' through
The money was just an opportunity
It's a chance I wanted
So do I succeed
Or do I take my lumps
I'm a man that never takes no for an answer
And I'll get to where I'm goin'
I won't take no for an answer
I'll take this right to the top
And I'm doin' it for the money
Yeah, I'm doin' it for the money
[Chorus]
I'm a man that never takes no for an answer
And I'll get to where I'm goin'
I won't take no for an answer
I'll take this right to the top
And I'm doin' it for the money
Yeah, I'm doin' it for the money
[Verse 2]
I've been waiting for my moment
Wish that I'd have gotten a bit more exposure
I coulda taken control
But things happened and shit that just got real
No one recognized me
I had to find my own way
But now I'm on the rise
Takin' over others
But my goal is still the same
I'll hit the mark if it's meant for me to do so
I've been through the grind
I've been through the heartache
But I've always known that I'll get there
I'll get there if I still believe
I haven't been myself in a long time
But now I'm back just getting all the shit done
I've been through the war
And I've got the scars
But I'm still comin' through
The money was just an opportunity
It's a chance I wanted
So do I succeed
Or do I take my lumps
I'm a man that never takes no for an answer
And I'll get to where I'm goin'
I won't take no for an answer
I'll take this right to the top
And I'm doin' it for the money
Yeah, I'm doin' it for the money
[Outro]
Success, success, success
So whatcha mean to me?
Whatcha mean to me?
Yeah, whatcha mean to me?
Tell me, whatcha mean to me?
Success, success, success
So whatcha mean to me?
Whatcha mean to me?
Yeah, whatcha mean to me?
Success, success, success
So whatcha mean to me?
Whatcha mean to me?
Yeah, whatcha mean to me?
Success, success, success
So whatcha mean to me?
Whatcha mean to me?
Yeah, whatcha mean to me?
Success, success, success
So whatcha mean to me?
Whatcha mean to me?
Yeah, whatcha mean to me?
Success,

"Write a song that explores the desire for success and the challenges faced along the way. The lyrics should convey a sense of determination, confidence, and a willingness to confront haters and doubters. The song should also touch on the themes of money, fame, and the pursuit of personal fulfillment.\n[Chorus]\nI'm a man that never takes no for an answer\nAnd I'll get to where I'm goin'\nI won't take no for an answer\nI'll take this right to the top\nAnd I'm doin' it for the money\nYeah, I'm doin' it for the money\n[Verse 1]\nI've been waiting for my moment\nWish that I'd have gotten a bit more exposure\nI coulda taken control\nBut things happened and shit that just got real\nNo one recognized me\nI had to find my own way\nBut now I'm on the rise\nTakin' over others\nBut my goal is still the same\nI'll hit the mark if it's meant for me to do so\nI've been through the grind\nI've been through the heartache\nBut I've always known that I'll get there\nI'll get there if I still believe\nI

[Chorus]
I'm a man that never takes no for an answer
And I'll get to where I'm goin'
I won't take no for an answer
I'll take this right to the top
And I'm doin' it for the money
Yeah, I'm doin' it for the money
[Verse 1]
I've been waiting for my moment
Wish that I'd have gotten a bit more exposure
I coulda taken control
But things happened and shit that just got real
No one recognized me
I had to find my own way
But now I'm on the rise
Takin' over others
But my goal is still the same
I'll hit the mark if it's meant for me to do so
I've been through the grind
I've been through the heartache
But I've always known that I'll get there
I'll get there if I still believe
I haven't been myself in a long time
But now I'm back just getting all the shit done
I've been through the war
And I've got the scars
But I'm still comin' through
The money was just an opportunity
It's a chance I wanted
So do I succeed
Or do I take my lumps

[Chorus]
I'm a man that never takes no for an answer
And I'll get to where I'm goin'
I won't take no for an answer
I'll take this right to the top
And I'm doin' it for the money
Yeah, I'm doin' it for the money
[Verse 2]
I've been waiting for my moment
Wish that I'd have gotten a bit more exposure
I coulda taken control
But things happened and shit that just got real
No one recognized me
I had to find my own way
But now I'm on the rise
Takin' over others
But my goal is still the same
I'll hit the mark if it's meant for me to do so
I've been through the grind
I've been through the heartache
But I've always known that I'll get there
I'll get there if I still believe
I haven't been myself in a long time
But now I'm back just getting all the shit done
I've been through the war
And I've got the scars
But I'm still comin' through
The money was just an opportunity
It's a chance I wanted
So do I succeed
Or do I take my lumps
I'm a man that never takes no for an answer
And I'll get to where I'm goin'
I won't take no for an answer
I'll take this right to the top
And I'm doin' it for the money
Yeah, I'm doin' it for the money
[Outro]
Success, success, success
So whatcha mean to me?
Whatcha mean to me?
Yeah, whatcha mean to me?
Tell me, whatcha mean to me?
Success, success, success
So whatcha mean to me?
Whatcha mean to me?
Yeah, whatcha mean to me?